In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import csv
import re

In [2]:
msg = 'NAV ILS RWY 27 LOC/GP OUT OF SERVICE'

In [3]:
abbrev_dict = {}

with open('data/Contractions (Clean).csv', mode='r') as infile:
    reader = csv.reader(infile)
    next(reader)
    abbrev_dict = {rows[0]:rows[1] for rows in reader}
    
def expandNotam(msg):
    new_string = re.sub(r'[^a-zA-Z]',' ',str(msg))
    new_msg = ''
    for wrd in new_string.split():        
        try:
            new_msg = new_msg + ' ' + abbrev_dict[wrd]
        except:
            new_msg = new_msg + ' ' + wrd
    return new_msg.lower().strip()

In [4]:
expandNotam(msg)

'navigation instrument landing system runway locally glide path out of service'

In [5]:
df4 = pd.read_csv('data/notam_20201027_pipes_noquotes.csv', 
                  on_bad_lines='skip', 
                  encoding='utf-16', 
                  parse_dates=['POSSIBLE_START_DATE', 'POSSIBLE_END_DATE', 'ISSUE_DATE', 'CANCELED_DATE'],
                  quoting=csv.QUOTE_NONE, 
                  engine="python", 
                  delimiter="|")

df4['MIN_ALT'] = df4['MIN_ALT'].replace('MSL', 0)
df4['MAX_ALT'] = df4['MAX_ALT'].replace('MSL', 0)
df4['MIN_ALT'] = pd.to_numeric(df4['MIN_ALT'], errors='coerce')
df4['MAX_ALT'] = pd.to_numeric(df4['MAX_ALT'], errors='coerce')

In [6]:
for x in df4['TEXT'].head(20):
    print(x)

AIRSPACE VOLK SOUTH MOA ACT 500FT UP TO BUT NOT INCLUDING FL180
AIRSPACE VOLK EAST MOA ACT 8000FT UP TO BUT NOT INCLUDING FL180
AIRSPACE FALLS 1 MOA ACT 500FT UP TO BUT NOT INCLUDING FL180
AIRSPACE FALLS 2 MOA ACT 500FT UP TO BUT NOT INCLUDING FL180
AIRSPACE VOLK WEST MOA ACT 100FT UP TO BUT NOT INCLUDING FL180
ILS SV 109.5 MHZ RWY 03 U/S
CYQY ILS 07 U/S
RWY 17/35 CLSD EXC XNG
AIRSPACE JACKAL MOA ACT 11000FT UP TO BUT NOT INCLUDING FL180
AIRSPACE JACKAL LOW MOA ACT 100FT-10999FT
AIRSPACE COLUMBUS 3 MOA ACT 8000FT UP TO BUT NOT INCLUDING FL180
AIRSPACE COLUMBUS 4 MOA ACT 10000FT UP TO BUT NOT INCLUDING FL180
AIRSPACE AR112L(E) ACT FL190-FL230
AIRSPACE AR112L(E) ACT FL190-FL230
AIRSPACE AR101(S) ACT FL260-FL290
AIRSPACE AR101(S) ACT FL260-FL290
AIRSPACE ABEL EAST MOA ACT 5000FT-12999FT
AIRSPACE AR20(SW) ACT FL240-FL280
OBST - CRANE IN AREA RADIUS 20M PSN 494110N0180622E.  HGT 30M AGL/283M AMSL. DAY MARKING
CYSB CRANE 1000 FT BEYOND THR 04 AND 1200 FT RIGHT RCL. 150 FT AGL 1293 MSL. NOT L

In [7]:
df4['TEXT'] = df4['TEXT'].apply(expandNotam)

In [8]:
for x in df4['TEXT'].head(20):
    print(x)

airspace volk south military operations area active foot up to but not including flight level
airspace volk east military operations area active foot up to but not including flight level
airspace falls military operations area active foot up to but not including flight level
airspace falls military operations area active foot up to but not including flight level
airspace volk west military operations area active foot up to but not including flight level
instrument landing system sv mhz runway intensity unknown (weather reports only) south or southern latitude
cyqy instrument landing system intensity unknown (weather reports only) south or southern latitude
runway closed except crossing
airspace jackal military operations area active foot up to but not including flight level
airspace jackal low military operations area active foot foot
airspace columbus military operations area active foot up to but not including flight level
airspace columbus military operations area active foot up to 

In [9]:
corpus = df4['TEXT'].values

count_vect = CountVectorizer(stop_words=stopwords.words('english'), lowercase=True)
x_counts = count_vect.fit_transform(corpus)
x_counts.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [10]:
tfidf_transformer = TfidfTransformer()
x_tfidf = tfidf_transformer.fit_transform(x_counts)

In [11]:
dimension = 8
sample_size = 100000

np.random.seed(232323)
sample = x_tfidf[np.random.choice(x_tfidf.shape[0], sample_size, replace=False), :]

lda = LDA(n_components = dimension)
lda_array = lda.fit_transform(sample)

In [12]:
topic_vec = lda.transform(x_tfidf)

df4['TOPIC'] = np.argmax(topic_vec, axis = 1)

In [13]:
import pickle

df4.to_pickle("data/allData.pkl")

In [14]:
import pyLDAvis
import pyLDAvis.sklearn
import warnings

warnings.filterwarnings('ignore')
pyLDAvis.enable_notebook()

In [15]:
pyLDAvis.sklearn.prepare(lda, x_tfidf, count_vect)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.179469  0.014481       1        1  19.761379
5     -0.235435 -0.104933       2        1  19.027187
2     -0.096413  0.196677       3        1  13.658944
0      0.088052 -0.149175       4        1  13.083363
6      0.094033 -0.085155       5        1  12.292405
4      0.101037  0.013151       6        1   9.351282
7      0.170448  0.089011       7        1   7.926025
1      0.057747  0.025944       8        1   4.899415, topic_info=            Term          Freq         Total Category  logprob  loglift
77371       time  48368.000000  48368.000000  Default  30.0000  30.0000
27619       foot  65892.000000  65892.000000  Default  29.0000  29.0000
54063     notamc  34156.000000  34156.000000  Default  28.0000  28.0000
18014       data  27347.000000  27347.000000  Default  27.0000  27.0000
75351     system  31631.000000  31631.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
72964      south    659.977032  25385.921818   Topic8  -6.0236  -0.6337
21776    eastern    711.550952  53190.674969   Topic8  -5.9484  -1.2981
4181    approach    569.446875  10419.309849   Topic8  -6.1712   0.1093
45433  longitude    629.815378  78280.415136   Topic8  -6.0704  -1.8066
72987   southern    548.640195  23984.524783   Topic8  -6.2084  -0.7617

[547 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
319        1  0.114207  absolute
319        2  0.141138  absolute
319        3  0.624043  absolute
319        4  0.009370  absolute
319        5  0.025024  absolute
...      ...       ...       ...
87433      4  0.006564       zoa
87433      5  0.006564       zoa
87433      6  0.006564       zoa
87433      7  0.091900       zoa
87433      8  0.866483       zoa

[3504 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 3, 1, 7, 5, 8, 2])

In [16]:
pyLDAvis.sklearn.prepare(lda, x_tfidf, count_vect, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.077835 -0.252129       1        1  19.761379
5     -0.132291 -0.276513       2        1  19.027187
2     -0.274227 -0.031530       3        1  13.658944
0     -0.150004  0.223299       4        1  13.083363
6      0.222320 -0.051219       5        1  12.292405
4      0.201231  0.101084       6        1   9.351282
7      0.083324  0.264583       7        1   7.926025
1     -0.028187  0.022426       8        1   4.899415, topic_info=            Term          Freq         Total Category  logprob  loglift
77371       time  48368.000000  48368.000000  Default  30.0000  30.0000
27619       foot  65892.000000  65892.000000  Default  29.0000  29.0000
54063     notamc  34156.000000  34156.000000  Default  28.0000  28.0000
18014       data  27347.000000  27347.000000  Default  27.0000  27.0000
75351     system  31631.000000  31631.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
72964      south    659.977032  25385.921818   Topic8  -6.0236  -0.6337
21776    eastern    711.550952  53190.674969   Topic8  -5.9484  -1.2981
4181    approach    569.446875  10419.309849   Topic8  -6.1712   0.1093
45433  longitude    629.815378  78280.415136   Topic8  -6.0704  -1.8066
72987   southern    548.640195  23984.524783   Topic8  -6.2084  -0.7617

[547 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
319        1  0.114207  absolute
319        2  0.141138  absolute
319        3  0.624043  absolute
319        4  0.009370  absolute
319        5  0.025024  absolute
...      ...       ...       ...
87433      4  0.006564       zoa
87433      5  0.006564       zoa
87433      6  0.006564       zoa
87433      7  0.091900       zoa
87433      8  0.866483       zoa

[3504 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 3, 1, 7, 5, 8, 2])

In [17]:
pyLDAvis.sklearn.prepare(lda, x_tfidf, count_vect, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3     -108.958252 -208.941483       1        1  19.761379
5      186.487061  -98.103249       2        1  19.027187
2       19.245832  160.858780       3        1  13.658944
0      170.658829   75.445076       4        1  13.083363
6       -1.386411  -35.043739       5        1  12.292405
4     -145.800095  106.944611       6        1   9.351282
7       64.834541 -221.825272       7        1   7.926025
1     -196.906036  -58.990913       8        1   4.899415, topic_info=            Term          Freq         Total Category  logprob  loglift
77371       time  48368.000000  48368.000000  Default  30.0000  30.0000
27619       foot  65892.000000  65892.000000  Default  29.0000  29.0000
54063     notamc  34156.000000  34156.000000  Default  28.0000  28.0000
18014       data  27347.000000  27347.000000  Default  27.0000  27.0000
75351     system  31631.000000  31631.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
72964      south    659.977032  25385.921818   Topic8  -6.0236  -0.6337
21776    eastern    711.550952  53190.674969   Topic8  -5.9484  -1.2981
4181    approach    569.446875  10419.309849   Topic8  -6.1712   0.1093
45433  longitude    629.815378  78280.415136   Topic8  -6.0704  -1.8066
72987   southern    548.640195  23984.524783   Topic8  -6.2084  -0.7617

[547 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
319        1  0.114207  absolute
319        2  0.141138  absolute
319        3  0.624043  absolute
319        4  0.009370  absolute
319        5  0.025024  absolute
...      ...       ...       ...
87433      4  0.006564       zoa
87433      5  0.006564       zoa
87433      6  0.006564       zoa
87433      7  0.091900       zoa
87433      8  0.866483       zoa

[3504 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 3, 1, 7, 5, 8, 2])

/Users/brensikeg/opt/anaconda3/envs/daen690/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/brensikeg/opt/anaconda3/envs/daen690/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/brensikeg/opt/anaconda3/envs/daen690/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/brensikeg/opt/anaconda3/envs/daen690/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/brensikeg/opt/anaconda3/envs/daen690/lib/pytho